In [1]:
from shutil import copyfile

def copy_files_to_folder(path, ids, result_path):
    all_files = glob.glob(path)
    for idi in ids:
        fis = glob.glob(path+"%s_*.yaml"%idi)
        for fi in fis:
            stem = fi.split("/")[-1]
            copyfile(fi, result_path + "/" + stem)

copy_files_to_folder("data/results/abc_00*/", rev_ext, "data/results/abc_revext")

NameError: name 'rev_ext' is not defined

In [2]:
from entity_mapper import EntityMapper
from topology_dict_builder import TopologyDictBuilder
#from geometry_dict_builder import GeometryDictBuilder
from utils import load_dictionary_from_file, write_dictionary_to_file
import glob
from pathlib import Path
import os
import meshplot as mp
import numpy as np

from yaml import CLoader
import yaml
from tqdm.auto import tqdm

def parse_topo(topo, geo, index=-1, filename=None):
    loops_closed = [0, 0]
    if index == -1:
        start_index = 0
        end_index = None
    else:
        start_index = index
        end_index = index+10
        
    # Iterate over parts
    pbar_parts = tqdm(list(topo["parts"]), disable=True)
    for pi, part in enumerate(pbar_parts):
        pbar_parts.set_description("Part %i/%i\r"%(pi, len(topo["parts"])))
        
        # Skip regions for now
        
        
        skippers = []
        
        # Iterate over shells
        pbar_shells = tqdm(list(part["shells"]), disable=True)
        for si, shell in enumerate(pbar_shells):
            pbar_shells.set_description("Shell %i/%i\r"%(si, len(part["shells"])))
            shell_orientation = shell["orientation_wrt_region"]
            
            # Iterate over faces
            #p = mp.plot(np.array([]), np.array([]), return_plot=True)
            
            for fi, shell_face in enumerate(shell["faces"][start_index:end_index]):
                #print("Face %i/%i\r"%(fi, len(shell["faces"])))
                face_orientation = shell_face["face_orientation_wrt_shell"]
                face_index = shell_face["face_index"]
                face = part["faces"][face_index]
                
                # Get surface
                surface_index = face["surface"]
                surface = geo["parts"][pi]["surfaces"][surface_index]
                surface_orientation = face["surface_orientation"]
                
                if surface["type"] != "BSpline":
                    print("Not converted", filename, pi)
                    skippers.append([filename, pi])
                    break
                
                #visualize_3dsurface(surface, None)
                
                
                p = None
                # Get loops
                for li in face["loops"]:
                    loop = part["loops"][li]
                    
                    loops_closed[1] += 1
#                    curves2d = []
                    start_point = end_point = None
                    wire_closed = True
                    for hei in loop["halfedges"]:
                        halfedge = part["halfedges"][hei]
                        he_orientation = halfedge["orientation_wrt_edge"]
                        edge = part["edges"][halfedge["edge"]]
                        curve3d = geo["parts"][pi]["3dcurves"][edge["3dcurve"]]
                        curve2d = geo["parts"][pi]["2dcurves"][halfedge["2dcurve"]]
                        
                        if curve2d["type"] != "BSpline" or curve3d["type"] != "BSpline":
                            print("Not converted", filename, pi)
                            skippers.append([filename, pi])
                            break
                        
                        points2d = evaluate_2dcurve(curve2d, he_orientation)
                        
                        if type(start_point) == type(None):
                            start_point = points2d[0]
                            
                        if type(end_point) != type(None):
                            #print("Matching curve end ", points2d[0], end_point, np.allclose(points2d[0], end_point))
                            wire_closed = wire_closed and np.allclose(points2d[0], end_point)
                        
                        end_point = points2d[-1]
                    

                        #p = visualize_2dcurve(curve2d, he_orientation, p)

                    #print("Matching wire end ", end_point, start_point, np.allclose(start_point, end_point))
                    if type(start_point) != type(None) and type(end_point) != type(None):
                        wire_closed = wire_closed and np.allclose(start_point, end_point)
                        #print("Wire of %i curves closed, %s"%(len(loop["halfedges"]),wire_closed))
                        loop["closed"] = wire_closed
                        if not wire_closed:
                            loops_closed[0] += 1
                        #print(li, wire_closed)
                        
                #halfedges = loops["halfedges"]
                
    return loops_closed
                

                    
                    
from geomdl import BSpline, NURBS, trimming

def evaluate_2dcurve(curve, orientation):
    crv2d = NURBS.Curve()
    crv2d.degree = curve["degree"]
    crv2d.ctrlpts = curve["poles"]
    crv2d.knotvector = curve["knots"]
    crv2d.weights = curve["weights"]
    crv2d.opt = ['reversed', not orientation]
    
    ps2d = crv2d.evalpts
    if crv2d.opt["reversed"]:
        ps2d.reverse()
    return np.array(ps2d)
        
#     if not p:
#         p = mp.plot(, return_plot=True)
#     else:
#         p.add_points(np.array(ps2d))
        
#     return p


# vertices = [
#     -10.0, -15.0, 0.0,
#     -10.0,   5.0, 0.0,
#      10.0,   5.0, 0.0,
#     -10.0, -15.0, 0.0,
#      10.0, -15.0, 0.0,
#     -10.0,   5.0, 0.0,
#       0.0,  15.0, 0.0,
#      10.0,   5.0, 0.0,
#      10.0, -15.0, 0.0
# ]

# colors = [
#     0xff,
#     0xffff,
#     0xff00ff,
#     0x00ffff,
#     0xffff00,
#     0xff00,
#     0xff0000,
#     0xff00ff,
#     0xffffff
# ]

# line = k3d.line(vertices, colors=colors, width=0.5)

# plot = k3d.plot(antialias=True)
# plot += line
# plot.display()


def visualize_3dsurface(surf, curves3d, plot):
    srv3d = NURBS.Surface()
    srv3d.degree_u = surf["u_degree"]
    srv3d.degree_v = surf["v_degree"]

    srv3d.ctrlpts_size_u = len(surf["poles"])
    srv3d.ctrlpts_size_v = len(surf["poles"][0])
    srv3d.ctrlpts = [item for sublist in surf["poles"] for item in sublist]

    srv3d.knotvector_u = surf["u_knots"]
    srv3d.knotvector_v = surf["v_knots"]

    srv3d.weights = [item for sublist in surf["weights"] for item in sublist]

    # Set evaluation delta
    srv3d.delta = 0.05#25

    # Evaluate surface points
    srv3d.evaluate()
    ps3ds = srv3d.evalpts
    ves = np.array(srv3d.vertices)
    fes = []
    for fss in range(len(srv3d.faces)):
        fes.append(srv3d.faces[fss].vertex_ids)
        
    mp.plot(np.array(ves), np.array(fes))
#     p3d = mp.plot(np.array(verts), np.array(faces), np.array(colors), shading={"wireframe": True}, return_plot=True)
#     #p3d = mp.plot(np.array(pts3ds), shading={"point_size": 0.5}, return_plot=True)
#     #p3d = mp.plot(np.array(ves), np.array(fes), shading={"wireframe": True}, return_plot=True)
#     p3d.add_points(np.array(pts3d[::5]), c=np.array(cls[::5]), shading={"point_size": 0.5})
#     for il, cl in enumerate(lines3ds):
#         p3d.add_lines(lines3ds[il][1:], lines3de[il][1:], shading={"line_color": cls_lines[il]})


#def visualize_2dsurface(loops, trim_doman, face_domain, surface_orientation):
    
    
                


def process(data_path="./results_abc_00nm", data_format="yaml"):
    data_dir = Path(data_path)
    
    # Glob files in data dir
    geo_files = sorted(data_dir.glob(f"*_geo.{data_format}"))
    topo_files = sorted(data_dir.glob(f"*_topo.{data_format}"))
    print(len(geo_files), len(topo_files))
    for idx, g in enumerate(geo_files[:]):
        #idi = g.stem.split("_")[0]
        idi = str(g).replace("_geo", "_topo")
        if idi != str(topo_files[idx]):
            print(idi,  topo_files[idx])
        #print(idi)
    assert len(geo_files) == len(topo_files)
    
    loops_closed = [0, 0]
    
    # Process files
    pbar = tqdm(range(len(geo_files)))
    for i in pbar:
        pbar.set_description("Processing %s"%topo_files[i])
        topo = load_dictionary_from_file(topo_files[i], data_format)
        geo = load_dictionary_from_file(geo_files[i], data_format)
        assert geo_files[i].stem.split("_")[0] == topo_files[i].stem.split("_")[0]
        
        loops_closed_topo = parse_topo(topo, geo, -1, geo_files[i])
        loops_closed[0] += loops_closed_topo[0]
        loops_closed[1] += loops_closed_topo[1]
        #if loops_closed / 100 
        print(loops_closed)#[0], loops_closed[1]*100)
        
        write_dictionary_to_file(topo_files[i], topo, data_format)
        

process(data_format='yaml')

ModuleNotFoundError: No module named 'entity_mapper'

In [10]:
all_ids = set(range(10000))

fails = sorted(list(all_ids.difference(set(success))))
print(fails)

[48, 414, 560, 700, 730, 787, 790, 959, 978, 988, 1996, 2501, 2550, 3087, 3094, 3483, 3752, 3807, 3902, 3946, 5410, 5587, 5605, 5641, 5642, 5687, 5759, 5760, 5786, 5987, 5988, 6679, 6741, 6747, 7501, 7744, 7894, 8001, 8287, 8660, 8836, 9272]


In [ ]:
Parts: Avg: 14.92
Times: Avg: 52.80
Errors occurred: 7(total)/6(single)
Conversion errors occurred: 2183(total)/974(single)
Success: 9958/10000
['699: Lines missing', '2508: Lines missing', '4147: Lines missing', '5047: Lines missing', '5802: Lines missing', '9109: Translation problem', '9109: 2020-12-30 23:26:11,258 ERROR Step transfer problem: 1\n']


In [1]:
import yaml
from yaml import CLoader
import meshplot as mp
import triangle

import numpy as np
from geomdl import BSpline, NURBS, trimming

# Import Matplotlib visualization module
from geomdl.visualization import VisMPL as vis
#from geomdl.visualization import VisVTK as vis


def load_features(path):
    with open(path, "r") as fi:
        features = yaml.load(fi, Loader=CLoader)
    return features

   
def visualize_topo(features, surf=-1, vis_3d=False, vis_2d=True):
    lines = []
    if surf == -1:
        start = 0
        end = len(features["topo"])
    else:
        start = surf
        end = surf+1
        
    for si, s in enumerate(features["topo"][start:end]):
 
        cpts = []
        #lines_s = []
        #lines_e = []
        #points = []
        #center = []
        curves2d = []
        curves3d = []

        s3d = features["surfaces"][si+start]
        surf_trim = s3d["trim_domain"]
        surf_face = s3d["face_domain"]
        #print(s.keys())
        surf_ori = bool(s["surf_orientation"])
        flip = False
        if "offsets" in s:
            print(s["offsets"], len(s["offsets"]), len(s["2dcurves"]))
           
            #if np.sum(np.array(s["offsets"])) > -0.0001:
            #    flip = False
            
            #print(s["surf_orientation"])
            #print(s["wire_orientations"], s["orientations"], s["wire_ids"])
            #for iii in s["orientations"]:
            #    print(bool(iii) != bool(s["surf_orientation"]))

        if s3d["type"] == "BSpline":
            srv3d = NURBS.Surface()
            srv3d.degree_u = s3d["u_degree"]
            srv3d.degree_v = s3d["v_degree"]

            srv3d.ctrlpts_size_u = len(s3d["poles"])
            srv3d.ctrlpts_size_v = len(s3d["poles"][0])
            srv3d.ctrlpts = [item for sublist in s3d["poles"] for item in sublist]

            srv3d.knotvector_u = s3d["u_knots"]
            srv3d.knotvector_v = s3d["v_knots"]

            srv3d.weights = [item for sublist in s3d["weights"] for item in sublist]

            # Set evaluation delta
            srv3d.delta = 0.05#25

            # Evaluate surface points
            #srv3d.evaluate()
            surf3d = srv3d

        for i in range(len(s["2dcurves"])):
            c3d = features["curves"][s["3dcurves"][i]]
            c2d = features["trim"][s["2dcurves"][i]]       

            #if c2d["type"] == "Line":
            #    loc = np.array(c2d["location"])
            #    ori = np.array(c2d["direction"])
            #    ivv = np.array(c2d["interval"])
            #    p1 = loc + ori*ivv[0]
            #    p2 = loc + ori*ivv[-1]
            if c2d["type"] == "BSpline":
                if "offsets" in s and len(s["offsets"]) > i:
                    offsets = np.array(s["offsets"][i])
                else:
                    print("Zero Offset missing")
                    offsets = np.zeros(2)
                p1 = np.array(c2d["poles"][0] + offsets)
                p2 = np.array(c2d["poles"][-1] + offsets)

                crv2d = NURBS.Curve()
                #crv2d.rational = 
                #print(c2d["rational"])
                crv2d.degree = c2d["degree"]
                poles = list(map(lambda x: (x[0] + offsets[0], x[1] + offsets[1]), c2d["poles"]))
                crv2d.ctrlpts = poles
                crv2d.knotvector = c2d["knots"]
                crv2d.weights = c2d["weights"]
                print(surf_ori, bool(s["wire_orientations"][s["wire_ids"][i]]), bool(s["orientations"][i]), bool(s["orientations"][i]) ^ (bool(s["surf_orientation"])))# ^ bool(s["wire_orientations"][s["wire_ids"][i]])))
                if flip:
                    crv2d.opt = ['reversed', bool(s["orientations"][i])]# ^ (not surf_ori)]
                else:
                    crv2d.opt = ['reversed', bool(s["orientations"][i])]# ^ surf_ori]


                curves2d.append(crv2d)

            if c3d["type"] == "BSpline":
                p1_3d = np.array(c3d["poles"][0])
                p2_3d = np.array(c3d["poles"][-1])

                crv3d = NURBS.Curve()
                #crv3d.rational = 
                #print(c3d["rational"])
                crv3d.degree = c3d["degree"]
                crv3d.ctrlpts = c3d["poles"]
                crv3d.knotvector = c3d["knots"]
                crv3d.weights = c3d["weights"]
                cpts.append(c3d["poles"])

                curves3d.append(crv3d)

            #p1[abs(p1)<1e-10] = 0.0
            #p2[abs(p2)<1e-10] = 0.0

            if s["orientations"][i] == 0:
                ps_2d = p1
                pe_2d = p2
                ps_3d = p1_3d
                pe_3d = p2_3d
            else:
                ps_2d = p2
                pe_2d = p1
                ps_3d = p2_3d
                pe_3d = p1_3d

            print(c2d["type"], "(", s["wire_ids"][i],") 2D: (%0.1f/%0.1f) -> (%0.1f/%0.1f) 3D: (%0.1f/%0.1f/%0.1f) -> (%0.1f/%0.1f/%0.1f) "%
                  (ps_2d[0], ps_2d[1], pe_2d[0], pe_2d[1], ps_3d[0], ps_3d[1], ps_3d[2], pe_3d[0], pe_3d[1], pe_3d[2]))
            #lines_s.append(p_s2d)
            #lines_e.append(p_e2d)
            #points.extend([p1, p2])
            #center.append((p1 + p2)/2.0)

        # 2D trimming curves
        lines2ds = [] # Trimming curves
        lines2de = []
        lines3ds = [] # 3D curves
        lines3de = []
        pts2d = [] # Evaluated trimming points
        pts3d = [] # Evaluated 3d points
        pts3dc = [] # 3D control points
        cls = [] # Colors for points
        cls_lines = [] # Colors for lines
        ins2d = []
        inscol = []
        edges2d = []
        ec = 0
        wires = sorted(list(set(s["wire_ids"])))
        c_id = 0
        #print(wires, s["wire_ids"])
        
        p_in = []
        p_out = []
        
        wire_closed = True
        for w in wires:
            ecw = 0 # edges in wire
            startp = endp = None
            for ic in range(c_id, c_id + s["wire_ids"].count(w)):
                c = curves2d[ic]
                ps2d = c.evalpts
                if c.opt["reversed"]:
                    ps2d.reverse()
                    
                if type(startp) == type(None):
                    startp = np.array(ps2d[0])
                
                if type(endp) != type(None):
                    print("Matching curve end ", np.array(ps2d[0]), endp, np.allclose(np.array(ps2d[0]), endp))
                    wire_closed = wire_closed and np.allclose(np.array(ps2d[0]), endp)
                    #print("Matching end ", ps2d[0], start, np.allclose(ps2d[0], start))

                endp = np.array(ps2d[-1])
                
                ps3d = curves3d[ic].evalpts
                lines2ds.append(np.array(ps2d))
                lines2de.append(np.roll(np.array(ps2d), 1, axis=0))
                lines3ds.append(np.array(ps3d))
                lines3de.append(np.roll(np.array(ps3d), 1, axis=0))
                if ic == c_id + s["wire_ids"].count(w) - 1:
                    wo = 1
                else:
                    wo = 0
                for iiec in range(ec, ec+len(ps2d)-wo):
                    edges2d.append([iiec, iiec+1])
                ec += len(ps2d)
                ecw += len(ps2d)
                pts2d.extend(ps2d)
                pts3d.extend(ps3d)
                pts3dc.extend(cpts[ic])
                col = np.random.rand(3)
                cls.extend([col]*len(ps2d))
                cls_lines.append('#%02x%02x%02x' % (int(col[0]*255), int(col[1]*255), int(col[2]*255)))
                
#                 if ic < len(s["points_in"]):
#                     p_in.append(s["points_in"][ic])
#                 if ic < len(s["points_out"]):
#                     p_out.append(s["points_out"][ic])

                #print("REVF", c.opt["reversed"])
                for ip, pp in enumerate(ps2d[:-1]):
                    d = np.array(ps2d[ip+1]) - np.array(ps2d[ip])
                    #if not c.opt["reversed"]:
                    rd = np.array([-d[1], d[0]])
                    #else:
                    #    rd = np.array([d[1], -d[0]])
                    ins2d.append(ps2d[ip] - rd)
                    inscol.append(col)
            
            print("Matching wire end ", endp, startp, np.allclose(startp, endp))
            wire_closed = wire_closed and np.allclose(startp, endp)
            print("Wire closed", wire_closed)
            edges2d.append([iiec+1, iiec+2-ecw])
            c_id += s["wire_ids"].count(w)

        for ic in range(len(s["points_in"])):
            p_in.append(s["points_in"][ic])
        for ic in range(len(s["points_out"])):
            p_out.append(s["points_out"][ic])
            
        #print(dir(surf3d))
        if False:
            from geomdl import tessellate

            surf3d.tessellator = tessellate.TrimTessellate()

            rs = [0, 0, 0]
            for iic, c in enumerate(curves2d):
                # = ['reversed', 1]
                c.opt = ["reversed", rs[iic]]
                #print(c.opt)
                #surf3d.add_trim(c)# curves2d
            #print(surf3d.trims)
            # Assuming that "surf" variable stores the surface instance

            surf3d.trims = curves2d
            trimming.fix_multi_trim_curves(surf3d)
            print(surf3d.tessellator.is_tessellated())
            surf3d.tessellate()
            print(surf3d.tessellator.is_tessellated())
        
        # 3D faces from NURBS "meshing"
        ps3ds = surf3d.evalpts
        #print(dir(s), len(s.faces), s.vertices)
        pts3ds = ps3ds
        ves = np.array(surf3d.vertices)
        fes = []
        for fss in range(len(surf3d.faces)):
            fes.append(surf3d.faces[fss].vertex_ids)
        #print(np.array(surf3d.faces[0].vertex_ids), surf3d.faces[0])
        
        #print(ves, fes)
        
        # 3D faces from OCC meshing
        verts = []
        faces = []
        colors = []
        v_cnt = 0
        f_cnt = 0
        fs = features["surfaces"][si+start]
        #print(fs["type"])
        for f in fs["faces"]:
            faces.append([f[0] + v_cnt, f[1] + v_cnt, f[2] + v_cnt])

        color = np.random.rand(1)
        color = np.array([0.5, 0.5, 0.5])
        for v in fs["verts"]:
            verts.append(v)
            colors.append(color)
            v_cnt += 1
            



        if vis_2d: 
            v = np.array(pts2d[::])
            idx = np.arange(v.shape[0]).reshape(v.shape[0], 1)
            e = np.hstack([idx, np.roll(idx, -1)])
            #print(e.shape)
            #print(v.shape)
            #e = np.array([[0, 1], [1, 2], [2, 3], [3, 0], [4, 5], [5, 6], [6, 7], [7, 4]], dtype=np.int32)
            h = np.array(ins2d[:1])
            #print(h)
            #print(np.array([ins2d::5]))
            p = mp.plot(v, shading={"point_size": 0.02}, return_plot=True)
            p.add_edges(v, np.array(edges2d))
            p.add_points(np.array(p_in), shading={"point_size": 0.2})
            p.add_points(np.array(p_out), shading={"point_size": 0.2, "point_color": "green"})
            
            edges2d = np.array(edges2d).astype(np.int32)
            #print(np.array(edges2d))
            #print(np.array(edges2d))
            #print(edges2d, e)

            
            A = {"vertices": v, "segments": edges2d, "holes": h}
            #t = triangle.triangulate(tri=A, opts="p")
            #ext = np.zeros((v.shape[0],1))
            #v = np.append(v, ext, axis=1)
            #print(t)
            #mp.plot(t["vertices"], t["triangles"], shading={"wireframe": True})
            
            
            p2d = mp.plot(np.array(pts2d[::5]), c=np.array(cls[::5]), shading={"point_size": 0.1}, return_plot=True)
            #print(np.array(cls).shape, np.array(pts2d).shape, len(lines2ds))
            p2d.add_points(np.array(ins2d[::5]), c=np.array(inscol[::5]), shading={"point_size": 0.1})
            for il, cl in enumerate(lines2ds):
                p2d.add_lines(lines2ds[il][1:], lines2de[il][1:], shading={"line_color": cls_lines[il]})
            #p = mp.plot(np.array(points), c="black", shading={"point_size": 2.0}, return_plot=True)
            #p.add_points(np.array(center), c=np.linspace(0.0, 1.0, len(center)), shading={"point_size": 1.0})
            #p.add_lines(np.array(lines_s), np.array(lines_e))
        
        if vis_3d:
            #print(type(ves), type(fes), ves.shape, fes.shape)#, vs, fs)
            p3d = mp.plot(np.array(verts), np.array(faces), np.array(colors), shading={"wireframe": True}, return_plot=True)
            #p3d = mp.plot(np.array(pts3ds), shading={"point_size": 0.5}, return_plot=True)
            #p3d = mp.plot(np.array(ves), np.array(fes), shading={"wireframe": True}, return_plot=True)
            p3d.add_points(np.array(pts3d[::5]), c=np.array(cls[::5]), shading={"point_size": 0.5})
            for il, cl in enumerate(lines3ds):
                p3d.add_lines(lines3ds[il][1:], lines3de[il][1:], shading={"line_color": cls_lines[il]})
            #p3dctl = mp.plot(np.array(verts), np.array(faces), np.array(colors), shading={"wireframe": True}, return_plot=True)
            #p3dctl.add_points(np.array(pts3dc), shading={"point_size": 0.5})
       
        
# features = load_features("00000044_features2_053.yml")
# snr = -1
# features = convert_features(features, snr=snr)
# with open("fi.yml", "w") as fi:
#     yaml.dump(features, fi)
# visualize_topo(features, surf=snr)

#features = load_features("00000044_c7d977f326364e35bb5b5d27_step_007_features_053_11.yml")
#visualize_topo(features, surf=8, p=p)



In [2]:
def convert_features(features, snr=-1):
    if snr == -1:
        start = 0
        end = len(features["topo"])
    else:
        start = snr
        end = snr+1
    tr_curves = features["curves"]
    trim_curves = features["trim"]
    #print(start, end, len(features["topo"]))
    for p_cnt, p in enumerate(features["topo"][start:end]):
        #print(p)
        p["offsets"] = [[0.0, 0.0]]
        wires = sorted(list(set(p["wire_ids"])))
        c_id = 0
        for w in wires:
            #*p["wire_ids"].count(w)]
            # Check orientation of first curve in wire
            if p["wire_ids"].count(w) >= 2:
                cur = tr_curves[p["3dcurves"][c_id]]
                nxt = tr_curves[p["3dcurves"][c_id+1]]
                c_ori = p["orientations"][c_id]
                n_ori = p["orientations"][c_id+1]
                if c_ori == 0:
                    pole0 = np.array(cur["poles"][0])
                    pole1 = np.array(cur["poles"][-1])
                else: 
                    pole0 = np.array(cur["poles"][-1])
                    pole1 = np.array(cur["poles"][0])

                if n_ori == 0:
                    pole2 = np.array(nxt["poles"][0])
                    pole3 = np.array(nxt["poles"][-1])
                else: 
                    pole2 = np.array(nxt["poles"][-1])
                    pole3 = np.array(nxt["poles"][0])

#                    print(pole0, pole1, pole2, pole3)

                d02 = np.linalg.norm(pole0 - pole2)
                d12 = np.linalg.norm(pole1 - pole2)
                d03 = np.linalg.norm(pole0 - pole3)
                d13 = np.linalg.norm(pole1 - pole3)

                amin = np.argmin([d02, d12, d03, d13]) 
                #print("S", p_cnt, c_id, amin, c_ori, n_ori, d02, d12, d03, d13, np.isclose(np.linalg.norm(pole0 - pole1), 0))

                #if np.min([d02, d12, d03, d13]) > 1e-7:
                    #print("Broken first curve", d02, d12, d03, d13, pole0, pole1, pole2, pole3)

                if (amin == 0 or amin == 2) and not np.isclose(np.linalg.norm(pole0 - pole1), 0): # Orientation of first curve incorrect, fix
                    if np.min([d02, d12, d03, d13]) <= 1e-7:
                        print("FIX ORIENT 0", p_cnt)
                        #print(d02, d12, d03, d13)
                        #print(pole0, pole1, pole2, pole3)
                        #print(amin, c_ori, n_ori, d02, d12, d03, d13)
                        p["orientations"][c_id] = abs(c_ori - 1)
                    else:
                        #print("NOT FIX 0", d02, d12, d03, d13)
                        pass

            # Fix all orientations in wire
            for i in range(c_id, c_id + p["wire_ids"].count(w) - 1):
                #print(i)
                cur = tr_curves[p["3dcurves"][i]]
                #print("P:", cur["poles"])
                nxt = tr_curves[p["3dcurves"][i+1]]
                c_ori = p["orientations"][i]
                n_ori = p["orientations"][i+1]
                if c_ori == 0:
                    pole1 = np.array(cur["poles"][-1])
                else: 
                    pole1 = np.array(cur["poles"][0])

                if n_ori == 0:
                    pole2 = np.array(nxt["poles"][0])
                    pole3 = np.array(nxt["poles"][-1])
                else: 
                    pole2 = np.array(nxt["poles"][-1])
                    pole3 = np.array(nxt["poles"][0])

                d12 = np.linalg.norm(pole1 - pole2)
                d13 = np.linalg.norm(pole1 - pole3)

                amin = np.argmin([d12, d13])
                #print(d12, d13, amin)
                #print("A", p_cnt, i, amin, c_ori, n_ori, d12, d13)

                if amin == 1: # Incorrect orientation, flip
                    if np.min([d12, d13]) <= 1e-7:
                        print("FIX ORIENT %i"%i, p_cnt)
                        #print(d12, d13, amin)

                        print(pole1, pole2, pole3, c_ori, n_ori, d12, d13, amin)
                        p["orientations"][i+1] = abs(n_ori - 1)
                    else:
                        #print("NOT FIX %i"%i, d12, d13)
                        pass
                        
            last_offset = np.array([0.0, 0.0])
            for i in range(c_id, c_id + p["wire_ids"].count(w) - 1):
                cur = trim_curves[p["2dcurves"][i]]
                nxt = trim_curves[p["2dcurves"][i+1]]
                c_ori = p["orientations"][i]
                n_ori = p["orientations"][i+1]
                if c_ori == 0:
                    pole1 = np.array(cur["poles"][-1]) + last_offset
                else: 
                    pole1 = np.array(cur["poles"][0]) + last_offset

                if n_ori == 0:
                    pole2 = np.array(nxt["poles"][0])
                    pole3 = np.array(nxt["poles"][-1])
                else: 
                    pole2 = np.array(nxt["poles"][-1])
                    pole3 = np.array(nxt["poles"][0])

                d12 = np.linalg.norm(pole1 - pole2)
                d13 = np.linalg.norm(pole1 - pole3)

                amin = np.argmin([d12, d13])
                #print(d12, d13, amin)
                #print("A", p_cnt, i, amin, c_ori, n_ori, d12, d13)

                if amin == 1: # Incorrect orientation, flip
                    if np.min([d12, d13]) <= 1e-7:
                        print("FIX ORIENT2D %i"%i, p_cnt)
                        #print(d12, d13, amin)

                        print(pole1, pole2, pole3, c_ori, n_ori, d12, d13, amin)
                        p["orientations"][i+1] = abs(n_ori - 1)
                    else:
                        #print("NOT FIX2 %i"%i, d12, d13)                        
                        pass

                if np.min([d12, d13]) > 1e-7:
                    if i < c_id + p["wire_ids"].count(w) - 2: # Curves afterwards in wire
                        nnxt = trim_curves[p["2dcurves"][i+2]]
                        nn_ori = p["orientations"][i+2]
                    else:
                        nnxt = trim_curves[p["2dcurves"][c_id]] # No curves afterwards, go back to first
                        nn_ori = p["orientations"][c_id]

                    if nn_ori == 0:
                        pole4 = np.array(nnxt["poles"][0])
                    else: 
                        pole4 = np.array(nnxt["poles"][-1])

                    #print("Broken int curve", i, d12, d13, pole1+last_offset, pole2, pole3, pole4)

                    ds = []
                    ps = [] #orientations
                    ofs = []
                    p1 = pole1
                    for ox in [-2*np.pi, -np.pi, -1.0, 0.0, 1.0, np.pi, 2*np.pi]:
                        for oy in [-2*np.pi, -np.pi, -1.0, 0.0, 1.0, np.pi, 2*np.pi]:

                            p2 = np.array([pole2[0]+ox, pole2[1]+oy])
                            p3 = np.array([pole3[0]+ox, pole3[1]+oy])
                            if np.abs(ox) > 3.0: # case of +-2pi
                                p2[0] = np.abs(p2[0])
                                p3[0] = np.abs(p3[0])
                            if np.abs(oy) > 3.0:
                                p2[1] = np.abs(p2[1])
                                p3[1] = np.abs(p3[1])


                            ds.append(np.linalg.norm(p1 - p2) + np.linalg.norm(pole4 - p3))
                            ds.append(np.linalg.norm(p1 - p3) + np.linalg.norm(pole4 - p2))
                            #ps.append([p2, p3])
                            #ps.append([p3, p2])
                            ps.extend([0, 1])
                            ofs.append([ox, oy])#, ox2, oy2])
                            ofs.append([ox, oy])
                    #dmin = np.argmin(ds)
                    dss = np.argsort(ds)
                    #nxt["poles"][0] = ps[dmin][0].tolist()
                    #nxt["poles"][1] = ps[dmin][1].tolist()
                    #print(tr_curves[p["2dcurves"][c_id+1]])
                    #print(np.sort(ds), dss, ofs[dss[0]], ofs[dss[1]], ofs[dss[2]], ofs[dss[3]])#, ps[dss[0]], ps[dss[1]])
                    #if np.sort(ds)[0] > 1e-7:
                    #    print("Probably wrong offset chosen.")
                    #print(ps[dss[0]], ds[dss[0]], ps[dss[1]], ds[dss[1]])
                    if ps[dss[0]] == 0 and ds[dss[0]] < 1e-6:
                        p["offsets"].append(ofs[dss[0]])
                        last_offset = ofs[dss[0]]
                        #print("c0")
                    elif ps[dss[1]] == 0 and ds[dss[1]] < 1e-6:
                        p["offsets"].append(ofs[dss[1]])
                        last_offset = ofs[dss[1]]
                        #print("c1")
                    else:
                        print("Probably wrong offset chosen.")
                        p["offsets"].append(np.array([0.0, 0.0]))
                        last_offset = np.array([0.0, 0.0])
                    #print("OA")
                else:
                    p["offsets"].append(np.array([0.0, 0.0]))
                    last_offset = np.array([0.0, 0.0])
                    #print("OFFs")
                    #print("Wire", p["wire_ids"])
                    #print("OO")


            c_id += p["wire_ids"].count(w)
        if len(p["2dcurves"]) > len(p["offsets"]):
            #print(p["offsets"])
            #print("Missing Offset")
            p["offsets"].append([0.0, 0.0])
        #print(p["offsets"])

    # Apply offset transformations
    if True:
        for si, s in enumerate(features["topo"]):
            #print(s["offsets"])
            for i in range(len(s["2dcurves"])):
                c2d = features["trim"][s["2dcurves"][i]]
                if c2d["type"] == "BSpline":
                    if "offsets" in s and len(s["offsets"]) > i:
                        s["offsets"][i] = np.array(s["offsets"][i]).tolist()
                    else:
                        #print("Zero Offset missing")
                        s["offsets"].append([0.0, 0.0])

                    #print("OFF", offsets)
                    #print("BEF", c2d["poles"])
                    #c2d["poles"] = list(map(lambda x: [(x[0] + offsets[0]).tolist(), (x[1] + offsets[1]).tolist()], c2d["poles"]))
                    #print("AFT", c2d["poles"])
            #del s["offsets"]

    return features

In [3]:
from OCC.Extend.TopologyUtils import TopologyExplorer, WireExplorer
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface, BRepAdaptor_CompCurve, BRepAdaptor_Curve2d
from OCC.Core.gp import *
from OCC.Core.BRepTools import *
from OCC.Core.BRep import *
from OCC.Core.TopoDS import *
import sys
import os
import numpy as np
import json
import glob
import fileinput
import random
import yaml
import igl
import copy
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepTopAdaptor import *
from OCC.Core.TopAbs import *
from OCC.Core.BRepBndLib import brepbndlib_Add
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeCylinder
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.STEPControl import STEPControl_Reader, STEPControl_Writer, STEPControl_AsIs
from OCC.Core.IFSelect import IFSelect_RetDone, IFSelect_ItemsByEntity
from OCC.Core.TColStd import TColStd_Array1OfReal, TColStd_Array2OfReal
from OCC.Core.TColgp import TColgp_Array1OfPnt, TColgp_Array2OfPnt, TColgp_Array1OfPnt2d
from OCC.Core.BRepGProp import (brepgprop_SurfaceProperties,
                                brepgprop_VolumeProperties)
from OCC.Core.Geom2dAdaptor import Geom2dAdaptor_Curve
from OCC.Core.GeomConvert import geomconvert_CurveToBSplineCurve, geomconvert_SurfaceToBSplineSurface
from OCCUtils.edge import Edge
from OCCUtils.Topology import Topo
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_NurbsConvert

from OCC.Core.GeomConvert import geomconvert_SurfaceToBSplineSurface
from OCC.Core.GeomConvert import geomconvert_CurveToBSplineCurve
from OCC.Core.Geom2dConvert import geom2dconvert_CurveToBSplineCurve
from OCC.Core.TopLoc import TopLoc_Location

np.set_printoptions(precision=17)

def read_step_file(filename, return_as_shapes=False, verbosity=False):
    assert os.path.isfile(filename)
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(filename)
    if status == IFSelect_RetDone:  # check status
        if verbosity:
            failsonly = False
            step_reader.PrintCheckLoad(failsonly, IFSelect_ItemsByEntity)
            step_reader.PrintCheckTransfer(failsonly, IFSelect_ItemsByEntity)
        shapes = []
        nr = 1
        try:
            while True:
                ok = step_reader.TransferRoot(nr)
                if not ok:
                    break
                _nbs = step_reader.NbShapes()
                shapes.append(step_reader.Shape(nr))  # a compound
                #assert not shape_to_return.IsNull()
                nr += 1
        except:
            print("No Shape", nr)
    else:
        raise AssertionError("Error: can't read file.")

    return shapes


def get_boundingbox(shape, tol=1e-6, use_mesh=True):
    bbox = Bnd_Box()
    bbox.SetGap(tol)
    if use_mesh:
        mesh = BRepMesh_IncrementalMesh()
        mesh.SetParallel(True)
        mesh.SetShape(shape)
        mesh.Perform()
        assert mesh.IsDone()
    brepbndlib_Add(shape, bbox, use_mesh)

    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    return [xmin, ymin, zmin, xmax, ymax, zmax, xmax-xmin, ymax-ymin, zmax-zmin]

edge_map = {0: "Line", 1: "Circle", 2: "Ellipse", 3: "Hyperbola", 4: "Parabola", 5: "Bezier", 6: "BSpline", 7: "Offset", 8: "Other"}
surf_map = {0: "Plane", 1: "Cylinder", 2: "Cone", 3: "Sphere", 4: "Torus", 5: "Bezier", 6: "BSpline", 7: "Revolution", 8: "Extrusion", 9: "Offset", 10: "Other"}
gmsh_map = {"Surface of Revolution": "Revolution", "Surface of Extrusion": "Extrusion", "Plane": "Plane", "Cylinder": "Cylinder",\
           "Cone": "Cone", "Torus": "Torus", "Sphere": "Sphere", "Bezier surface": "Bezier", "BSpline surface": "BSpline", "Unknown": "Other"}


def convert_curve(curve):
    d1_feat = {"type": edge_map[curve.GetType()]}
    c_type = d1_feat["type"]
    if c_type == "Line":
        c = curve.Line()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["direction"] = list(c.Direction().Coord())
        scale_factor = 1000.0
    elif c_type == "Circle":
        c = curve.Circle()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["z_axis"] = list(c.Axis().Direction().Coord())
        d1_feat["radius"] = c.Radius()
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
        scale_factor = 1.0
    elif c_type == "Ellipse":
        c = curve.Ellipse()
        d1_feat["focus1"] = list(c.Focus1().Coord())
        d1_feat["focus2"] = list(c.Focus2().Coord())
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
        d1_feat["z_axis"] = list(c.Axis().Direction().Coord())
        d1_feat["maj_radius"] = c.MajorRadius()
        d1_feat["min_radius"] = c.MinorRadius()
        scale_factor = 1.0
    elif c_type == "BSpline":
        c = curve.BSpline()
        c.SetNotPeriodic()
        d1_feat["rational"] = c.IsRational()
        d1_feat["closed"] = c.IsClosed()
        d1_feat["continuity"] = c.Continuity()
        d1_feat["degree"] = c.Degree()
        p = TColgp_Array1OfPnt(1, c.NbPoles())
        c.Poles(p)
        points = []
        for pi in range(p.Length()):
            points.append(list(p.Value(pi+1).Coord()))
        d1_feat["poles"] = points

        k = TColStd_Array1OfReal(1, c.NbPoles() + c.Degree() + 1)
        c.KnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d1_feat["knots"] = knots

        w = TColStd_Array1OfReal(1, c.NbPoles())
        c.Weights(w)
        weights = []
        for wi in range(w.Length()):
            weights.append(w.Value(wi+1))
        d1_feat["weights"] = weights

        scale_factor = 1.0
    else:
        print("Unsupported type 3d", c_type)

    return d1_feat

def convert_2dcurve(curve, convert_2bs=False):
    curve_t = curve.Edge()
    d1_feat = {"type": edge_map[curve.GetType()], "interval": [curve.FirstParameter(), curve.LastParameter()]}        
    c_type = d1_feat["type"]

    if c_type == "Line":
        c = curve.Line()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["direction"] = list(c.Direction().Coord())
    elif c_type == "Circle":
        c = curve.Circle()
        d1_feat["location"] = list(c.Location().Coord())
        d1_feat["radius"] = c.Radius()
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
    elif c_type == "Ellipse":
        c = curve.Ellipse()
        d1_feat["focus1"] = list(c.Focus1().Coord())
        d1_feat["focus2"] = list(c.Focus2().Coord())
        d1_feat["x_axis"] = list(c.XAxis().Direction().Coord())
        d1_feat["y_axis"] = list(c.YAxis().Direction().Coord())
        d1_feat["maj_radius"] = c.MajorRadius()
        d1_feat["min_radius"] = c.MinorRadius()
    elif c_type == "BSpline":
        if not convert_2bs:
            c = curve.BSpline()
        else:
            c = curve
        c.SetNotPeriodic()
        d1_feat["rational"] = c.IsRational()
        d1_feat["closed"] = c.IsClosed()
        d1_feat["continuity"] = c.Continuity()
        d1_feat["degree"] = c.Degree()
        p = TColgp_Array1OfPnt2d(1, c.NbPoles())
        c.Poles(p)
        points = []
        for pi in range(p.Length()):
            points.append(list(p.Value(pi+1).Coord()))
        d1_feat["poles"] = points

        k = TColStd_Array1OfReal(1, c.NbPoles() + c.Degree() + 1)
        c.KnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d1_feat["knots"] = knots

        w = TColStd_Array1OfReal(1, c.NbPoles())
        c.Weights(w)
        weights = []
        for wi in range(w.Length()):
            weights.append(w.Value(wi+1))
        d1_feat["weights"] = weights
    else:
        print("Unsupported type 2d", c_type)
    return d1_feat

def convert_surface(surf, convert_2bs=False):
    surf_t = surf.Face()
    d2_feat = {"type": surf_map[surf.GetType()]}

    s_type = d2_feat["type"]
        
    if s_type == "Plane":
        s = surf.Plane()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())

    elif s_type == "Cylinder":
        s = surf.Cylinder()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())
        d2_feat["radius"] = s.Radius()

    elif s_type == "Cone":
        s = surf.Cone()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())
        d2_feat["radius"] = s.RefRadius()
        d2_feat["angle"] = s.SemiAngle()
        d2_feat["apex"] = list(s.Apex().Coord())

    elif s_type == "Sphere":
        s = surf.Sphere()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["coefficients"] = list(s.Coefficients())
        d2_feat["radius"] = s.Radius()

    elif s_type == "Torus":
        s = surf.Torus()
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Axis().Direction().Coord())
        d2_feat["x_axis"] = list(s.XAxis().Direction().Coord())
        d2_feat["y_axis"] = list(s.YAxis().Direction().Coord())
        d2_feat["max_radius"] = s.MajorRadius()
        d2_feat["min_radius"] = s.MinorRadius()


    elif s_type == "Bezier":
        print("BEZIER SURF")

    elif s_type == "BSpline":
        if not convert_2bs:
            c = surf.BSpline()
        else:
            c = surf

        _round = lambda x: round(x, 15)
        d2_feat["trim_domain"] = list(map(_round, breptools_UVBounds(surf_t)))
        d2_feat["face_domain"] = list(map(_round, c.Bounds()))
        d2_feat["is_trimmed"] = d2_feat["trim_domain"] != d2_feat["face_domain"]
        #print(c.IsUPeriodic(), c.IsVPeriodic())
        c.SetUNotPeriodic()
        c.SetVNotPeriodic()
        #print(d2_feat["trim_domain"], d2_feat["face_domain"])
        d2_feat["u_rational"] = c.IsURational()
        d2_feat["v_rational"] = c.IsVRational()
        d2_feat["u_closed"] = c.IsUClosed()
        d2_feat["v_closed"] = c.IsVClosed()
        d2_feat["continuity"] = c.Continuity()
        d2_feat["u_degree"] = c.UDegree()
        d2_feat["v_degree"] = c.VDegree()

        p = TColgp_Array2OfPnt(1, c.NbUPoles(), 1, c.NbVPoles())
        c.Poles(p)
        points = []
        for pi in range(p.ColLength()):
            elems = []
            for pj in range(p.RowLength()):
                elems.append(list(p.Value(pi+1, pj+1).Coord()))
            points.append(elems)
        d2_feat["poles"] = points

        k = TColStd_Array1OfReal(1, c.NbUPoles() + c.UDegree() + 1)
        c.UKnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d2_feat["u_knots"] = knots

        k = TColStd_Array1OfReal(1, c.NbVPoles() + c.VDegree() + 1)
        c.VKnotSequence(k)
        knots = []
        for ki in range(k.Length()):
            knots.append(k.Value(ki+1))
        d2_feat["v_knots"] = knots

        w = TColStd_Array2OfReal(1, c.NbUPoles(), 1, c.NbVPoles())
        c.Weights(w)
        weights = []
        for wi in range(w.ColLength()):
            elems = []
            for wj in range(w.RowLength()):
                elems.append(w.Value(wi+1, wj+1))
            weights.append(elems)
        d2_feat["weights"] = weights

        scale_factor = 1.0

    elif s_type == "Revolution":
        s = surf.AxeOfRevolution()
        c = surf.BasisCurve()
        d1_feat = convert_curve(c)
        d2_feat["location"] = list(s.Location().Coord())
        d2_feat["z_axis"] = list(s.Direction().Coord())
        d2_feat["curve"] = d1_feat

    elif s_type == "Extrusion":
        c = surf.BasisCurve()
        d1_feat = convert_curve(c)
        d2_feat["direction"] = list(surf.Direction().Coord())
        d2_feat["curve"] = d1_feat
        
    else:
        print("Unsupported type", s_type)
    
    return d2_feat

def setup_trimming(face, bounds, tol=1e-12):
    u1,u2, v1,v2 = bounds
    dailen = (u2-u1)*(u2-u1) + (v2-v1)*(v2-v1)
    dailen = np.sqrt(dailen) / 400.
    tol = max([dailen, tol])
    return BRepTopAdaptor_FClass2d(face, tol), dailen


def is_uv_in_face_domain(trim, u, v):
    uv = gp_Pnt2d(u, v)
    if trim.Perform(uv) == TopAbs_IN:
        return True
    else:
        return False

occ_topoe = None
    
def mesh_model(model, res_path, convert=True, all_edges=True, snr=0, mmi=0):
    global occ_topoe
    fil = model.split("/")[-1][:-5]
    folder = "/".join(model.split("/")[:-1])
    with fileinput.FileInput(model, inplace=True) as fi:
        for line in fi:
            print(line.replace("UNCERTAINTY_MEASURE_WITH_UNIT( LENGTH_MEASURE( 1.00000000000000E-17 )",
                           "UNCERTAINTY_MEASURE_WITH_UNIT( LENGTH_MEASURE( 1.00000000000000E-17 )"), end='')

    occ_steps = read_step_file(model)
    bt = BRep_Tool()

    for occ_cnt in range(mmi, mmi+1):#len(occ_steps)):#
        if convert:
            try:
                nurbs_converter = BRepBuilderAPI_NurbsConvert(occ_steps[occ_cnt])
                nurbs_converter.Perform(occ_steps[occ_cnt])
                nurbs = nurbs_converter.Shape()
            except Exception as e:
                print("Conversion failed")
                #print(occ_steps[occ_cnt])
                #surfi = occ_steps[occ_cnt]
                #if 'Cylinder'in str(e):
                print(e.args, str(e))
                #    return surfi
                continue
        else:
            nurbs = occ_steps[occ_cnt]
            
        mesh = BRepMesh_IncrementalMesh(occ_steps[occ_cnt], 0.9, False, 0.5, True)
        mesh.Perform()
        if not mesh.IsDone():
            print("Mesh is not done.")
            continue
        
        occ_topo = TopologyExplorer(nurbs)
        occ_topoe = occ_topo
        occ_top = Topo(nurbs)
        occ_topo1 = TopologyExplorer(occ_steps[occ_cnt])
        occ_top1 = Topo(occ_steps[occ_cnt])
        
        d1_feats = []
        d2_feats = []
        t_curves = []
        tr_curves = []
        stats = {}
        stats["model"] = model
        total_edges = 0
        total_surfs = 0
        stats["curves"] = []
        stats["surfs"] = []
        c_cnt = 0
        t_cnt = 0

        # Iterate over edges
        for edge in occ_topo.edges():
            curve = BRepAdaptor_Curve(edge)
            stats["curves"].append(edge_map[curve.GetType()])
            d1_feat = convert_curve(curve)
            
            if edge_map[curve.GetType()] == "Other":
                continue
            
            for f in occ_top.faces_from_edge(edge):
                if f == None:
                    print("Broken face")
                    continue
                su = BRepAdaptor_Surface(f)
                c = BRepAdaptor_Curve2d(edge, f)
                t_curve = {"surface": f, "3dcurve": edge, "3dcurve_id": c_cnt, "2dcurve_id": t_cnt}
                t_curves.append(t_curve)
                tr_curves.append(convert_2dcurve(c))
                t_cnt += 1

            d1_feats.append(d1_feat)
            c_cnt += 1
            total_edges += 1

        patches = []
        faces1 = list(occ_topo1.faces())
        # Iterate over faces
        for fci, face in enumerate(occ_topo.faces()):
            surf = BRepAdaptor_Surface(face)
            #print(dir(surf.Surface()), dir(face))
            stats["surfs"].append(surf_map[surf.GetType()])
            d2_feat = convert_surface(surf)
            
            if surf_map[surf.GetType()] == "Other":
                continue

            for tc in t_curves:
                if tc["surface"] == face:
                    patch = {"3dcurves": [], "2dcurves": [], "orientations": [], "surf_orientation": face.Orientation(),
                            "wire_ids": [], "wire_orientations": [], "points_in": [], "points_out": []}
                    
                    for wc, fw in enumerate(occ_top.wires_from_face(face)):
                        patch["wire_orientations"].append(fw.Orientation())
                        if all_edges:
                            edges = [i for i in WireExplorer(fw).ordered_edges()]
                        else:
                            edges = list(occ_top.edges_from_wire(fw))
                        for fe in edges:
                            for ttc in t_curves:
                                if ttc["3dcurve"].IsSame(fe) and tc["surface"] == ttc["surface"]:
                                    patch["3dcurves"].append(ttc["3dcurve_id"])
                                    patch["2dcurves"].append(ttc["2dcurve_id"])
                                    patch["wire_ids"].append(wc)
                                    orientation = fe.Orientation()
                                    patch["orientations"].append(orientation)
                                    
                                    bnds = [surf.FirstUParameter(), surf.LastUParameter(), surf.FirstVParameter(), surf.LastVParameter()]
                                    trim, diag = setup_trimming(face, bnds)
                                    cur = tr_curves[ttc["2dcurve_id"]]
                                    
#                                     print(bnds)
                                    u_dist = bnds[1]-bnds[0]
                                    v_dist = bnds[3]-bnds[2]
                                    samples_u = np.linspace(bnds[0]-0.01*u_dist, bnds[1]+0.01*u_dist, int(10*u_dist))
                                    samples_v = np.linspace(bnds[2]-0.01*v_dist, bnds[3]+0.01*v_dist, int(10*v_dist))
                                    #print(samples_u, samples_v)

#                                     for u in samples_u:
#                                         for v in samples_v:
#                                             inuv = is_uv_in_face_domain(trim, u, v)
#                                             if inuv:
#                                                 patch["points_in"].append([float(u), float(v)])
#                                                 #print("In", u, v)
#                                             else:
#                                                 #print("Out", u, v)
#                                                 patch["points_out"].append([float(u), float(v)])
                                            
#                                     idi = int(len(cur["poles"])/2)
#                                     diri = np.array(cur["poles"][idi+1]) - np.array(cur["poles"][idi])
#                                     diril = np.linalg.norm(np.array(cur["poles"][idi+1]) - np.array(cur["poles"][idi]))
#                                     diri /= np.linalg.norm(diri)
#                                     print(diri)
#                                     step = 100.0
#                                     for stepi in range(100):
#                                         l = step * diril
#                                         p1 = l * np.array([-diri[1], diri[0]]) + np.array(cur["poles"][idi])
#                                         p2 = l * np.array([diri[1], -diri[0]]) + np.array(cur["poles"][idi]) 
#                                         p1_in = is_uv_in_face_domain(trim, p1[0], p1[1])
#                                         p2_in = is_uv_in_face_domain(trim, p2[0], p2[1])
#                                         #print(p1_in, p2_in)
#                                         if p1_in and not p2_in:
#                                             patch["points_in"].append(p1.tolist())
#                                             patch["points_out"].append(p2.tolist())
#                                             break

#                                         if p2_in and not p1_in:
#                                             patch["points_in"].append(p2.tolist())
#                                             patch["points_out"].append(p1.tolist())
#                                             break
                                        
#                                         step /= 2.0
                                        
#                                         if stepi == 9:
#                                             print(np.array(cur["poles"][idi]), l * np.array([-diri[1], diri[0]]) + np.array(cur["poles"][idi]))
#                                             print("Fucked up")
#                                             patch["points_in"].append(p1.tolist())
#                                             patch["points_out"].append(p2.tolist())

#                                         val1 = gp_Pnt()
#                                         f.D0(u, v, val1)
#                                         verts.append((val1.X(), val1.Y(), val1.Z()))
#                                         colors.append(fi)
                                      
                    patches.append(patch)
                    break
            
            location = TopLoc_Location()
            facing = (bt.Triangulation(faces1[fci], location))
            if facing != None:
                tab = facing.Nodes()
                tri = facing.Triangles()
                verts = []
                for i in range(1, facing.NbNodes()+1):
                    verts.append(list(tab.Value(i).Coord()))

                faces = []
                for i in range(1, facing.NbTriangles()+1):
                    index1, index2, index3 = tri.Value(i).Get()
                    faces.append([index1 - 1, index2 - 1, index3 - 1])
            
                os.makedirs(res_path, exist_ok=True)
                #igl.write_triangle_mesh("%s/%s_%03i_mesh_%04i.obj"%(res_path, fil, occ_cnt, fci), np.array(verts), np.array(faces))
                d2_feat["faces"] = faces
                d2_feat["verts"] = verts
            else:
                print("Missing triangulation")
                continue
            
            d2_feats.append(d2_feat)
            total_surfs += 1
            
            
        bbox =  get_boundingbox(occ_steps[occ_cnt], use_mesh=False)
        xmin, ymin, zmin, xmax, ymax, zmax = bbox[:6]
        bbox1 = ["%.2f"%xmin, "%.2f"%ymin, "%.2f"%zmin, "%.2f"%xmax, "%.2f"%ymax, "%.2f"%zmax, "%.2f"%(xmax-xmin), "%.2f"%(ymax-ymin), "%.2f"%(zmax-zmin)]
        stats["#edges"] = total_edges
        stats["#surfs"] = total_surfs
        
        #print(patches[0]) 
        #if len(p["curves"]) > 
        features = {"curves": d1_feats, "surfaces": d2_feats, "trim": tr_curves, "topo": patches, "bbox": bbox1}
        features = convert_features(features, snr=snr)            
        
        #print(patches)
        #print(len(features["surfaces"]), features["topo"][0])
        #visualize_topo(features, surf=snr)
        #visualize_features(features, surf=snr)
        
        os.makedirs(res_path, exist_ok=True)
        fip = fil + "_features2"
        with open("%s/%s_%03i.yml"%(res_path, fip, occ_cnt), "w") as fili:
            yaml.dump(features, fili, indent=2)
        
#         fip = fil + "_features"
#         with open("%s/%s_%03i.yml"%(res_path, fip, occ_cnt), "w") as fili:
#             features2 = copy.deepcopy(features)
#             for sf in features2["surfaces"]:
#                 del sf["faces"]
#                 del sf["verts"]
#             yaml.dump(features2, fili, indent=2)    

#        res_path = folder.replace("/step/", "/stat/")
#        fip = fil + "_stats"
#        with open("%s/%s_%03i.yml"%(res_path, fip, occ_cnt), "w") as fili:
#            yaml.dump(stats, fili, indent=2)
    
    print("Writing results for %s with %i parts."%(model, len(occ_steps)))


In [4]:
import glob
import time
fs = sorted(glob.glob("data/conv/*/*.step"))
print(len(fs))
#t1 = time.time()
for f in fs[9000:]:
    #print(f)
    comp = glob.glob("res/%s*"%f.split("/")[-1].split("_")[0])
    if len(comp) == 1:
        print("Contained %s"%f)
    else:
        print("Processing %s"%f)
        surfi = mesh_model(f, "res", convert=True, all_edges=True, snr=-1, mmi=-1)
#print(time.time()-t1)

# def mm_para(f):
#     print("Processing %f"%f)
#     mesh_model(f, "res", convert=True, all_edges=True, snr=-1, mmi=-1)

# from joblib import Parallel, delayed
# Parallel(n_jobs=10)(delayed(mm_para)(fi) for fi in fs)

10000
Contained data/conv/00009000/00009000_42eaa6e3ad6f4bb0901861b4_step_000.step
Processing data/conv/00009001/00009001_2e26094f45eb4b908b320c47_step_000.step
Conversion failed
('Standard_RangeError\nNCollection_Array1::Create\nwrapper details:\n  * symname: new_BRepBuilderAPI_NurbsConvert\n  * wrapname: _wrap_new_BRepBuilderAPI_NurbsConvert__SWIG_1\n  * fulldecl: BRepBuilderAPI_NurbsConvert::BRepBuilderAPI_NurbsConvert(TopoDS_Shape const &,Standard_Boolean const)',) Standard_RangeError
NCollection_Array1::Create
wrapper details:
  * symname: new_BRepBuilderAPI_NurbsConvert
  * wrapname: _wrap_new_BRepBuilderAPI_NurbsConvert__SWIG_1
  * fulldecl: BRepBuilderAPI_NurbsConvert::BRepBuilderAPI_NurbsConvert(TopoDS_Shape const &,Standard_Boolean const)
Writing results for data/conv/00009001/00009001_2e26094f45eb4b908b320c47_step_000.step with 2 parts.
Contained data/conv/00009002/00009002_f85d4dda0c184332be3f2d6b_step_000.step
Contained data/conv/00009003/00009003_d4a801ea45374c9886ffa3a5_

In [ ]:
import triangle
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt
v = np.array([[-1.0, -1.0], [1, -1], [1, 1], [-1, 1], [-2, -2], [2, -2], [2, 2], [-2, 2]])
v = np.array([[-2, -2], [2, -2], [2, 2], [-2, 2]])

e = np.array([[0, 1], [1, 2], [2, 3], [3, 0]], dtype=np.int32)
h = np.array([[0.0, 0.0]])

A = {"vertices": v, "segments": e}#, "holes": h}
t = triangle.triangulate(tri=A, opts="qp")
#ext = np.zeros((v.shape[0],1))
#v = np.append(v, ext, axis=1)
mp.plot(t["vertices"], t["triangles"], shading={"wireframe": True})
#print(t["vertices"].shape, t["vertex_markers"].shape, np.max(t["triangles"]))
triangle.compare(plt, A, t)
plt.show()

In [16]:
surfi = mesh_model("data/sphere.step", "ress", convert=True, all_edges=True, snr=-1, mmi=-1)

Unsupported type 3d Other
Unsupported type 3d Other
['AxeOfRevolution', 'BSpline', 'BasisCurve', 'BasisSurface', 'Bezier', 'Cone', 'Cylinder', 'D0', 'D1', 'D2', 'D3', 'DN', 'Direction', 'FirstUParameter', 'FirstVParameter', 'GetType', 'IsUClosed', 'IsUPeriodic', 'IsURational', 'IsVClosed', 'IsVPeriodic', 'IsVRational', 'LastUParameter', 'LastVParameter', 'Load', 'NbUIntervals', 'NbUKnots', 'NbUPoles', 'NbVIntervals', 'NbVKnots', 'NbVPoles', 'OffsetValue', 'Plane', 'Sphere', 'Surface', 'Torus', 'UContinuity', 'UDegree', 'UIntervals', 'UPeriod', 'UResolution', 'UTrim', 'VContinuity', 'VDegree', 'VIntervals', 'VPeriod', 'VResolution', 'VTrim', 'Value', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destr

NameError: name 'convert_features' is not defined

In [ ]:
print(surfi)
#nurbs_converter = BRepBuilderAPI_NurbsConvert(surfi)
#nurbs_converter.Perform(surfi)
occ_topo = TopologyExplorer(surfi)
occ_top = Topo(surfi)

for f in occ_topo.faces():
    surf = BRepAdaptor_Surface(f)
    if surf_map[surf.GetType()] == "Cylinder":
        d2_feat = convert_surface(surf)
        _round = lambda x: round(x, 15)
        c = surf.Cylinder()
        #print(dir(c))
        domain = list(map(_round, breptools_UVBounds(surf.Face())))
        print((domain[1] - domain[0]), 2 * np.pi)
        nurbs_converter = BRepBuilderAPI_NurbsConvert(f)
        nurbs_converter.Perform(f)
        #print(list(map(_round, surf.Cylinder().Bounds())))

In [ ]:
from geomdl import NURBS
from geomdl.visualization import VisVTK as vis

features2 = load_features("results/00000044_features2_053.yml")


# Generate surface
surf = NURBS.Surface()

bsurf = features2["surfaces"][-1]

# Set degrees
surf.degree_u = bsurf["u_degree"]
surf.degree_v = bsurf["v_degree"]

# Set control points
surf.ctrlpts_size_u = len(bsurf["poles"])
surf.ctrlpts_size_v = len(bsurf["poles"][0])
surf.ctrlpts = [item for sublist in bsurf["poles"] for item in sublist]

# Set knot vectors
surf.knotvector_u = bsurf["u_knots"]
surf.knotvector_v = bsurf["v_knots"]

# BSpline to NURBS and set weights
surf.weights = [item for sublist in bsurf["weigRevolutionhts"] for item in sublist]

# Set evaluation delta
surf.delta = 0.025

# Evaluate surface points
surf.evaluate()

# Plot the control point grid and the evaluated surface
vis_comp = vis.VisSurface()
surf.vis = vis_comp
surf.render()
pass

In [ ]:
from geomdl import BSpline
from geomdl import multi
from geomdl import knotvector

features2 = load_features("results/00000044_features2_053.yml")

curves = []
for s in features2["topo"][-1:]:
    for i in range(len(s["2dcurves"])):
        c3d = features2["trim"][s["2dcurves"][i]]
        #print(c3d.keys())


        crv = BSpline.Curve()
        crv.degree = c3d["degree"]
        #print(c3d["poles"])
        crv.ctrlpts = c3d["poles"]
        crv.knotvector = c3d["knots"]
        
        curves.append(crv)
        
mcrv = multi.CurveContainer(*curves)

# Import Matplotlib visualization module
from geomdl.visualization import VisMPL as vis
#from geomdl.visualization import VisVTK as vis

# Set the visualization component of the curve container
mcrv.vis = vis.VisCurve3D()

# Plot the curves in the curve container
mcrv.render()
pass

In [ ]:
# Set control points
crv1.ctrlpts = [[1, 0, 0], [1, 1, 0], [0, 1, 0]]

# Generate a uniform knot vector
crv1.knotvector = knotvector.generate(crv1.degree, crv1.ctrlpts_size)

# Create the curve instance #2
crv2 = BSpline.Curve()

# Set degree
crv2.degree = 3

# Set control points
crv2.ctrlpts = [[1, 0, 0], [1, 1, 0], [2, 1, 0], [1, 1, 0]]

# Generate a uniform knot vector
crv2.knotvector = knotvector.generate(crv2.degree, crv2.ctrlpts_size)

# Create a curve container
mcrv = multi.CurveContainer(crv1, crv2)

# Import Matplotlib visualization module
from geomdl.visualization import VisMPL

# Set the visualization component of the curve container
mcrv.vis = VisMPL.VisCurve3D()

# Plot the curves in the curve container
mcrv.render()

In [ ]:
import yaml
from yaml import CLoader as Loader, CDumper as Dumper
import numpy as np
import meshplot as mp
import igl
import glob

def read_model(obj_path, feat_path):
    m = {}
    m["vertices"], _, m["normals"], m["face_indices"], _, m["normal_indices"] = igl.read_obj(obj_path)
            
    with open(feat_path) as fi:
        m["features"] = yaml.load(fi, Loader=Loader)
    return m

def visualize(obj, feat):
    m = read_model(obj, feat)
    #print(m["vertices"].shape, m["normals"].shape, m["face_indices"].shape, m["normal_indices"].shape)

    # Generate vertex 2 normal map
    vert2norm = {}
    for fi in range(m["face_indices"].shape[0]):
        #print(f)
        for fii in range(3):
            if m["face_indices"][fi, fii] in vert2norm:
                vert2norm[m["face_indices"][fi, fii]].append(m["normal_indices"][fi, fii])
            else:
                vert2norm[m["face_indices"][fi, fii]] = [m["normal_indices"][fi, fii]]

    # Calculate sharp features
    nr_curves = len(m["features"]["curves"])
    nr_misclass_noabs = 0
    nr_misclass_thres = 0
    for d1c, d1f in enumerate(m["features"]["curves"]):
        sharp_noabs = True
        sharp_thres = True
        for vi in d1f["vert_indices"][1:-1]:
            nos = list(set(vert2norm[vi]))
            if len(nos) == 2:
                n0 = np.array(m["normals"][nos[0]])
                n1 = np.array(m["normals"][nos[1]])
                #print(n0, n1)
                #print(np.linalg.norm(n0), np.linalg.norm(n1))
                if n0.dot(n1) > 0.95:
                    sharp_noabs = False
                if n0.dot(n1) > 0.97:
                    sharp_thres = False
            else:
                sharp_noabs = False
                sharp_thres = False

        if d1f["sharp"] != sharp_noabs:
            #print("Update curve with noabs %i from %s to %s."%(d1c, d1f["sharp"], sharp_noabs))
            nr_misclass_noabs += 1

        if d1f["sharp"] != sharp_thres:
            #print("Update curve with thres %i from %s to %s."%(d1c, d1f["sharp"], sharp_thres))
            nr_misclass_thres += 1
            
        d1f["sharp_noabs"] = sharp_noabs
        d1f["sharp_thres"] = sharp_thres

    return nr_curves, nr_misclass_noabs, nr_misclass_thres

    # Colors for patches
    c = np.zeros(m["face_indices"].shape[0])
    for i, fe in enumerate(m["features"]["surfaces"]):
        for j in fe["face_indices"]:
            c[j] = i

    # Generate lines from sharp features
    lines = []
    lines_noabs = []
    lines_thres = []
    for i, fe in enumerate(m["features"]["curves"]):
        if fe["sharp"]:
            for j in range(len(fe["vert_indices"])-1):
                lines.append([fe["vert_indices"][j], fe["vert_indices"][j+1]])
        if fe["sharp_noabs"]:
            for j in range(len(fe["vert_indices"])-1):
                lines_noabs.append([fe["vert_indices"][j], fe["vert_indices"][j+1]])
        if fe["sharp_thres"]:
            for j in range(len(fe["vert_indices"])-1):
                lines_thres.append([fe["vert_indices"][j], fe["vert_indices"][j+1]])

    # Retrieve the normals
    if False:
        normals_s = [[], [], [], [], []]
        normals_e = [[], [], [], [], []]
        for vi in sorted(vert2norm.keys()):
            for i, ni in enumerate(list(set(vert2norm[vi]))):
                #print(i, vi, ni, list(set(vert2norm[vi])))
                normals_s[i].append(m["vertices"][vi])
                normals_e[i].append(m["vertices"][vi] + m["normals"][ni] * 0.1)

    # Visualize the sharp features
    if nr_misclass_noabs != 0 or nr_misclass_thres != 0:
        p = mp.plot(m["vertices"], m["face_indices"], c, return_plot=True)
        p.add_edges(m["vertices"], np.array(lines), shading={"line_color": "red", "line_width": 2.5});

        #p1 = mp.plot(m["vertices"], m["face_indices"], c, return_plot=True)
        #p1.add_edges(m["vertices"], np.array(lines_noabs), shading={"line_color": "red", "line_width": 2.5});

        p2 = mp.plot(m["vertices"], m["face_indices"], c, return_plot=True)
        p2.add_edges(m["vertices"], np.array(lines_thres), shading={"line_color": "red", "line_width": 2.5});

        
    
    # Visualize the normals
    if False:
        p.add_lines(np.array(normals_s[0]), np.array(normals_e[0]), shading={"line_color": "red"})
        p.add_lines(np.array(normals_s[1]), np.array(normals_e[1]), shading={"line_color": "black"})
        p.add_lines(np.array(normals_s[2]), np.array(normals_e[2]), shading={"line_color": "green"})
        p.add_lines(np.array(normals_s[3]), np.array(normals_e[3]), shading={"line_color": "blue"});
    
    return nr_curves, nr_misclass_noabs, nr_misclass_thres


#total = [0, 0, 0]
#interesting = ["00210457", "00210244", "00210325", 
#               "00210856", "00210566", "00210760", "00210381"]

for i in range(0):#len(feats)):
    if objs[i].split("/")[7] in interesting:
        cur, mis_abs, mis_thr = visualize(objs[i], feats[i])
        #total[0] += cur
        #total[1] += mis_abs
        #total[2] += mis_thr
        print(objs[i].split("/")[7], cur, mis_abs, mis_thr)

In [ ]:
for fii in ["00", "01"]:
    print("Processing %s"%fii)
    feats = sorted(glob.glob("/media/sebastian/Data/ABC/feat/abc_00%s_feat_v00/*/*.yml"%fii))
    objs = sorted(glob.glob("/media/sebastian/Data/ABC/obj/abc_00%s_obj_v00/*/*.obj"%fii))
    total = [0, 0, 0]
    broken_abs = []
    broken_thr = []
    for i in range(len(feats)):
        if i%100 == 0:
            print("Processed %i/%i"%(i, len(feats)))
        cur, mis_abs, mis_thr = visualize(objs[i], feats[i])
        total[0] += cur
        total[1] += mis_abs
        total[2] += mis_thr
        if mis_abs > 0:
            broken_abs.append("%s/%s %i/%i"%(objs[i].split("/")[7], objs[i].split("/")[8], mis_abs, cur))
        if mis_thr > 0:
            broken_thr.append("%s/%s %i/%i"%(objs[i].split("/")[7], objs[i].split("/")[8], mis_thr, cur))            
        #print(objs[i].split("/")[7], cur, mis_abs, mis_thr)
    print(total, total[1]/total[0], total[2]/total[0])
    with open("%s_abs.txt"%fii, "w") as fi:
        for b in broken_abs:
            fi.write(b)
            fi.write("\n")
    with open("%s_thr.txt"%fii, "w") as fi:
        for b in broken_thr:
            fi.write(b)
            fi.write("\n")
    print(len(broken_abs), len(broken_thr), len(feats))

In [ ]:
total[2]/total[0]